<a href="https://colab.research.google.com/github/elangbijak4/blockchain-codes-use-generative-AI/blob/main/Rev4_Infrastruktur_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import time
import json
import random

In [2]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = f"{index}{previous_hash}{timestamp}{data}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", int(time.time()), "Genesis Block", Block.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def to_dict(self):
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "hash": self.hash
        }

In [3]:
class ProofOfStake:
    def __init__(self):
        self.stakeholders = {}  # To keep track of stakeholder's stakes

    def add_stakeholder(self, address, stake):
        if address in self.stakeholders:
            self.stakeholders[address] += stake
        else:
            self.stakeholders[address] = stake

    def select_validator(self):
        total_stake = sum(self.stakeholders.values())
        pick = random.uniform(0, total_stake)
        current = 0
        for address, stake in self.stakeholders.items():
            current += stake
            if current > pick:
                return address

In [4]:
class Blockchain:
    def __init__(self):
        self.chain = [Block.create_genesis_block()]
        self.pos = ProofOfStake()

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        new_block = Block(index, previous_hash, timestamp, data, "")
        validator = self.pos.select_validator()
        print(f"Block #{index} created by {validator} using PoS")
        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)

    def add_stakeholder(self, address, stake):
        self.pos.add_stakeholder(address, stake)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.hash != Block.calculate_hash(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data):
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def to_json(self):
        return json.dumps([block.to_dict() for block in self.chain], indent=4)

In [5]:
class SmartContract:
    def __init__(self, code):
        self.code = code
        self.storage = {}

    def execute(self, blockchain, **kwargs):
        local_context = {
            'blockchain': blockchain,
            'storage': self.storage
        }
        local_context.update(kwargs)
        exec(self.code, {}, local_context)
        self.storage = local_context['storage']

In [6]:
class ERC721:
    def __init__(self):
        self.tokens = {}
        self.token_owners = {}
        self.token_metadata = {}
        self._token_owner = {}  # tokenId => owner
        self._owned_tokens = {}  # owner => list of tokenIds
        self._token_approvals = {}  # tokenId => approved address
        self._operator_approvals = {}  # owner => (operator => approved)

    def _mint(self, to, tokenId, metadata):
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to
        self.token_metadata[tokenId] = metadata

    def _transfer(self, from_, to, tokenId):
        self._owned_tokens[from_].remove(tokenId)
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to

    def mint(self, to, metadata):
        tokenId = hashlib.sha256(metadata.encode()).hexdigest()
        self._mint(to, tokenId, metadata)
        return tokenId

    def transfer(self, from_, to, tokenId):
        self._transfer(from_, to, tokenId)

In [7]:
class FungibleToken:
    def __init__(self):
        self.balances = {}
        self.total_supply = 0

    def mint(self, to, amount):
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.total_supply += amount

    def transfer(self, from_, to, amount):
        if from_ not in self.balances or self.balances[from_] < amount:
            raise ValueError("Insufficient balance")
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.balances[from_] -= amount

    def balance_of(self, address):
        return self.balances.get(address, 0)

In [8]:
class MyBlockchain(Blockchain):
    def __init__(self):
        super().__init__()
        self.smart_contracts = {}
        self.nft_contract = ERC721()
        self.fungible_token_contract = FungibleToken()

    def store_smart_contract(self, code):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        smart_contract = SmartContract(code)
        new_block = Block(index, previous_hash, timestamp, code, "")
        validator = self.pos.select_validator()
        print(f"Smart Contract Block #{index} created by {validator} using PoS")
        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        block_hash = new_block.hash
        self.smart_contracts[block_hash] = smart_contract
        return block_hash

    def execute_smart_contract_from_block(self, block_hash, **kwargs):
        if block_hash in self.smart_contracts:
            smart_contract = self.smart_contracts[block_hash]
            smart_contract.execute(self, **kwargs)
        else:
            print("Smart contract not found in blockchain")

    def mint_nft(self, owner, metadata):
        tokenId = self.nft_contract.mint(owner, metadata)
        data = json.dumps({
            "type": "mint",
            "owner": owner,
            "tokenId": tokenId,
            "metadata": metadata
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_nft(self, from_, to, tokenId):
        self.nft_contract.transfer(from_, to, tokenId)
        data = json.dumps({
            "type": "transfer",
            "from": from_,
            "to": to,
            "tokenId": tokenId
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def mint_fungible_token(self, to, amount):
        self.fungible_token_contract.mint(to, amount)
        data = json.dumps({
            "type": "mint_fungible",
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_fungible_token(self, from_, to, amount):
        self.fungible_token_contract.transfer(from_, to, amount)
        data = json.dumps({
            "type": "transfer_fungible",
            "from": from_,
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

In [9]:
# Contoh Penggunaan
my_blockchain = MyBlockchain()

In [10]:
my_blockchain.add_stakeholder('Alice', 10)
my_blockchain.add_stakeholder('Bob', 20)
my_blockchain.add_stakeholder('Charlie', 30)

In [11]:
# Menambahkan blok biasa
my_blockchain.add_block('Block 1 Data')
my_blockchain.add_block('Block 2 Data')

Block #1 created by Bob using PoS
Block #2 created by Charlie using PoS


In [12]:
# Menambahkan smart contract
smart_contract_code = """
def contract_function(storage):
    storage['counter'] = storage.get('counter', 0) + 1
    print(f"Counter value: {storage['counter']}")
contract_function(storage)
"""

In [13]:
block_hash = my_blockchain.store_smart_contract(smart_contract_code)
print(f"Smart contract stored in block with hash: {block_hash}")

Smart Contract Block #3 created by Bob using PoS
Smart contract stored in block with hash: 23d77737c0680aa6bef4b615710282cd11b04873d555fefa762d05a3957f9854


In [14]:
# Menjalankan smart contract dari blockchain
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 1


In [15]:
# Menjalankan kembali smart contract untuk melihat perubahan state
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 2


In [16]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1718695603,
        "data": "Genesis Block",
        "hash": "61a5f68d6c6a9a6c0173e99551c552c01205fe52afa466555489af972d2bbe97"
    },
    {
        "index": 1,
        "previous_hash": "61a5f68d6c6a9a6c0173e99551c552c01205fe52afa466555489af972d2bbe97",
        "timestamp": 1718695608,
        "data": "Block 1 Data",
        "hash": "e63a328247b866ba94cab3c88ea6357608236cf842958e1973d5899b29e6c62d"
    },
    {
        "index": 2,
        "previous_hash": "e63a328247b866ba94cab3c88ea6357608236cf842958e1973d5899b29e6c62d",
        "timestamp": 1718695608,
        "data": "Block 2 Data",
        "hash": "1fddadbcd6c1b1ae35d41003d4eb2dd0478aafb9ff5596e6436117b24adec000"
    },
    {
        "index": 3,
        "previous_hash": "1fddadbcd6c1b1ae35d41003d4eb2dd0478aafb9ff5596e6436117b24adec000",
        "timestamp": 1718695616,
        "data": "\ndef contract_function(storage):\n    st

In [17]:
# Membuat NFT baru
owner_address = "0xfeedfacefeedfacefeedfacefeedfacefeedface"
asset_uri = "https://example2.co.id"
mint_block_hash = my_blockchain.mint_nft(owner_address, asset_uri)
print(f"NFT minted in block with hash: {mint_block_hash}")

Block #4 created by Bob using PoS
NFT minted in block with hash: f09dcd3b2e8d87407c6b7d2216028c2766e733accf17f32cf7f3218ac9e745f4


In [18]:
# Transfer NFT
new_owner_address = "0xbadc0ffeebadc0ffeebadc0ffeebadc0ffeebadc"
token_id = hashlib.sha256(asset_uri.encode()).hexdigest()
transfer_block_hash = my_blockchain.transfer_nft(owner_address, new_owner_address, token_id)
print(f"NFT transferred in block with hash: {transfer_block_hash}")

Block #5 created by Bob using PoS
NFT transferred in block with hash: 4a917036fe82eb3c01e158e9cdbc0dc1d1efb025a00bcbcacbcaff73a449ed95


In [19]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1718695603,
        "data": "Genesis Block",
        "hash": "61a5f68d6c6a9a6c0173e99551c552c01205fe52afa466555489af972d2bbe97"
    },
    {
        "index": 1,
        "previous_hash": "61a5f68d6c6a9a6c0173e99551c552c01205fe52afa466555489af972d2bbe97",
        "timestamp": 1718695608,
        "data": "Block 1 Data",
        "hash": "e63a328247b866ba94cab3c88ea6357608236cf842958e1973d5899b29e6c62d"
    },
    {
        "index": 2,
        "previous_hash": "e63a328247b866ba94cab3c88ea6357608236cf842958e1973d5899b29e6c62d",
        "timestamp": 1718695608,
        "data": "Block 2 Data",
        "hash": "1fddadbcd6c1b1ae35d41003d4eb2dd0478aafb9ff5596e6436117b24adec000"
    },
    {
        "index": 3,
        "previous_hash": "1fddadbcd6c1b1ae35d41003d4eb2dd0478aafb9ff5596e6436117b24adec000",
        "timestamp": 1718695616,
        "data": "\ndef contract_function(storage):\n    st

In [20]:
# Demonstrasi Token Fungible (ERC20)
my_blockchain.mint_fungible_token("0xabc", 1000)
my_blockchain.mint_fungible_token("0xdef", 500)

Block #6 created by Bob using PoS
Block #7 created by Charlie using PoS


'1e5ae86081a7e9c89022d3ba9e9d29d3313afe9f68928293a604cdc06e373b0b'

In [21]:
print("Balance of 0xabc:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc: 1000
Balance of 0xdef: 500


In [22]:
my_blockchain.transfer_fungible_token("0xabc", "0xdef", 200)

Block #8 created by Charlie using PoS


'9947fde0f6639c89889bda1a111d605def8ca06755ca2711cd9502b93094cd0b'

In [23]:
print("Balance of 0xabc after transfer:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef after transfer:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc after transfer: 800
Balance of 0xdef after transfer: 700


In [24]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1718695603,
        "data": "Genesis Block",
        "hash": "61a5f68d6c6a9a6c0173e99551c552c01205fe52afa466555489af972d2bbe97"
    },
    {
        "index": 1,
        "previous_hash": "61a5f68d6c6a9a6c0173e99551c552c01205fe52afa466555489af972d2bbe97",
        "timestamp": 1718695608,
        "data": "Block 1 Data",
        "hash": "e63a328247b866ba94cab3c88ea6357608236cf842958e1973d5899b29e6c62d"
    },
    {
        "index": 2,
        "previous_hash": "e63a328247b866ba94cab3c88ea6357608236cf842958e1973d5899b29e6c62d",
        "timestamp": 1718695608,
        "data": "Block 2 Data",
        "hash": "1fddadbcd6c1b1ae35d41003d4eb2dd0478aafb9ff5596e6436117b24adec000"
    },
    {
        "index": 3,
        "previous_hash": "1fddadbcd6c1b1ae35d41003d4eb2dd0478aafb9ff5596e6436117b24adec000",
        "timestamp": 1718695616,
        "data": "\ndef contract_function(storage):\n    st